In [1]:
import pandas as pd
import requests
import time
import os
import json

In [18]:
# # Remove corrupted data (Magesium)
# list_files = os.listdir("../../data/imports")

# list_files_to_remove = []
# for file in list_files:
#     if file.upper().find("MAGNESIUM") > -1:
#         # os.unlink(f"../../data/imports/{file}")
#         list_files_to_remove.append(file)
#         # pass

In [2]:
# read all downloaded files
list_files = os.listdir("../../data/imports")
list_files = [tuple(x.split(".csv")[0].split("_")[1:]) for x in list_files]
list_files = set(list_files)
len(list_files)

127097

In [3]:
df_exports = pd.read_csv("../../data/hhi_data/df_hhi_exports.csv")
countries_list = json.loads(df_exports[["reporterCode", "reporterDesc"]].drop_duplicates().to_json(orient='records')) #columns, split, records

classification_codes = {
    "Aluminium": 2606,

    "Antimony": 261710,
    "Asbestos": 2524,
    "Barytes": 2511,
    "Bismuth": 8106,
    "Cadmium": 8107,
    "Chromium": 2610,
    "Coal": 2701,
    "Cobalt": 810520,
    "Copper": 2603,
    "Gold": 7108,
    "Graphite": 2504,
    "Iron": 2601,
    "Lead": 2607,
    "Lithium": 283691,
    "Magnesite": 251910,

    "Magnesium": 810411,
    # MISSING BAUXITE
    # "Bauxite": ?,

    "Manganese": 2602,
    "Mercury": 280540,
    "Molybdenum": 2613,
    "Natural gas": 271111,
    "Nickel": 2604,
    "Petroleum": 2709,
    "Rare earth": 2846,
    "Silver": 261610,
    "Tin": 2609,
    "Titanium": 2614,
    "Tungsten": 2611,
    "Uranium": 261210,
    "Zinc": 2608,
    "Zirconium": 261510,
    "Lithium-ion batteries": 850760 # igual que Litio?
}
classification_codes = {key.upper(): val for key, val in classification_codes.items()}
classification_codes_inverted = {val: key.upper() for key, val in classification_codes.items()}

years = range(2000, 2021)

In [4]:
# Get all possible combinations


possible_list_files = list()

for year in years:
    for commodity, commodity_code in classification_codes.items():
        for country in countries_list:
            possible_list_files.append((f'{year}', commodity, country['reporterDesc']))

possible_list_files = set(possible_list_files)
len(possible_list_files)


129024

In [5]:
# get rest of files to download
files_to_download = list(possible_list_files - list_files)
len(files_to_download)

1927

In [14]:
# Save total files into 4 equal parts
import pickle
indx = 0
size_list = len(files_to_download) // 4
for i in range(0, len(files_to_download), size_list):
    with open(f"list_files/files_to_download_{indx}.txt", "wb") as fp:
        pickle.dump(files_to_download[i:i + size_list], fp)
    indx += 1

### Get distinct countries/codes in the export file

In [2]:
import pickle
# Read n-list
indx = 4
with open(f"list_files/files_to_download_{indx}.txt", "rb") as fp:
    sublist_files_to_download = pickle.load(fp)

print("Len of list 0: ", len(sublist_files_to_download))


# indx = 3
# with open(f"list_files/files_to_download_{indx}.txt", "rb") as fp:
#     sublist_files_to_download += pickle.load(fp)

# print("Len of list [0] + [1]: ", len(sublist_files_to_download))


Len of list 0:  3


In [6]:
# classification_codes["INDIA"]
def get_country_code(country):
    for country_info in countries_list:
        if country.upper() == country_info["reporterDesc"].upper():
            return country_info["reporterCode"]
    return -1

In [7]:
# Only for last step
sublist_files_to_download = files_to_download

for year, commodity, country in sublist_files_to_download:
    try:
        country_code = get_country_code(country)
        url = f"https://comtradeplus.un.org/api/Trade/getTrade?" \
                f"selectedProductOptionsModified=C&" \
                f"selectedFrequencyOptionsModified=A&" \
                f"selectedClassificationOptionsModified=HS&" \
                f"selectValuePeriodsModified={year}&" \
                f"selectValueReportersModified={country_code}&" \
                f"selectValuePartnersModified=all&" \
                f"selectValueTradeflowsModified=m&" \
                f"selectValueCommodityCodesModified={classification_codes[commodity]}&" \
                f"selectValueCustomsCodesModified=c00&" \
                f"selectValueTransportCodesModified=0&" \
                f"selectValueSecondPartnersModified=0&" \
                f"selectValueAggregateByModified=none&" \
                f"selectValueBreakdownModeModified=plus&" \
                f"selectValueincludeDescModified=True&" \
                f"selectValuecountOnlyModified=False"
        
        payload={}
        headers = {
            'Accept': 'application/json, text/plain, */*',
            'Referer': f'https://comtradeplus.un.org/TradeFlow?Frequency=A&Flows=X&CommodityCodes={classification_codes[commodity]}&Partners=0&Reporters=all&period={year}&AggregateBy=none&BreakdownMode=plus'
        }

        try:
            response = requests.get(url=url, headers=headers, data=payload)
            json = response.json()
            pd.json_normalize(json["data"]).to_csv(f"../../data/imports/imports_{year}_{commodity}_{country}.csv", index=False)
        except Exception as e:
            print(f"[ERROR][{country}][{year}][{commodity_code}] Request: {e}")

    except Exception as e:
        print(f"[ERROR][TOP] {year}, {commodity}, {commodity_code}, {country}: [{e}]")

    # time.sleep(3)
        

In [ ]:
# # Lista de elementos de interés



# for country in countries_list:
#     for year in years:
#         for commodity, commodity_code in classification_codes.items():            
#             url = f"https://comtradeplus.un.org/api/Trade/getTrade?" \
#                 f"selectedProductOptionsModified=C&" \
#                 f"selectedFrequencyOptionsModified=A&" \
#                 f"selectedClassificationOptionsModified=HS&" \
#                 f"selectValuePeriodsModified={year}&" \
#                 f"selectValueReportersModified={country['reporterCode']}&" \
#                 f"selectValuePartnersModified=all&" \
#                 f"selectValueTradeflowsModified=m&" \
#                 f"selectValueCommodityCodesModified={commodity_code}&" \
#                 f"selectValueCustomsCodesModified=c00&" \
#                 f"selectValueTransportCodesModified=0&" \
#                 f"selectValueSecondPartnersModified=0&" \
#                 f"selectValueAggregateByModified=none&" \
#                 f"selectValueBreakdownModeModified=plus&" \
#                 f"selectValueincludeDescModified=True&" \
#                 f"selectValuecountOnlyModified=False"


#             payload={}
#             headers = {
#                 'Accept': 'application/json, text/plain, */*',
#                 'Referer': f'https://comtradeplus.un.org/TradeFlow?Frequency=A&Flows=X&CommodityCodes={commodity_code}&Partners=0&Reporters=all&period={year}&AggregateBy=none&BreakdownMode=plus'
#             }

#             try:
#                 response = requests.get(url=url, headers=headers, data=payload)
#                 json = response.json()
#                 pd.json_normalize(json["data"]).to_csv(f"../../data/imports/imports_{year}_{commodity}_{country['reporterDesc']}.csv", index=False)
#             except Exception as e:
#                 print(f"[ERROR][{year}][{commodity_code}] Request: {e}")

#             time.sleep(5)
            

## Create a conformed table for imports

In [9]:
root_path_imports = "../../data/imports"
list_files = os.listdir(root_path_imports)
len(list_files)

129024

In [ ]:
# dd = pd.read_csv(f"{root_path_imports}/imports_2000_ALUMINIUM_United Arab Emirates.csv")
# valid_rows = (~dd["qtyUnitAbbr"].isna()) | (dd["partnerDesc"].str.lower() == "world")
# valid_rows

In [24]:
# Create a list of Df and identify empty CSVs
years, commodities, countries = [], [], []
list_df = []
for file in list_files:
    try:
        df = pd.read_csv(f"{root_path_imports}/{file}")

        # # Remove NANs from "qtyUnitAbbr" iff it's not "WORDL"
        # valid_rows = (~df["qtyUnitAbbr"].isna()) | (df["partnerDesc"].str.lower() == "world")


        # Consider a dataframe where "world" is not included and check if it has more that one unit
        df_noWorld = df[df["partnerDesc"].str.lower() != "world"]
        # Remove NANs
        df_noWorld = df_noWorld[~df_noWorld["qtyUnitAbbr"].isna()]
        # Check if units are kilogram
        if len(df_noWorld["qtyUnitAbbr"].unique()) > 1:
            print(f"File: {file}")
            break
            raise ValueError(f"There are more units to consider: {df['qtyUnitAbbr'].unique()}")

        list_df.append(df)
    except Exception as e:
        print(f"[Error] {e}")
        metadata = file.split(".csv")[0].split("_")[1:]
        years.append(metadata[0])
        commodities.append(metadata[1])
        countries.append(metadata[2])
        

# From the list, create only one file
final_imports_df = pd.concat(list_df, ignore_index=True)

# Create cSV with all the information of empty dataframes
empty_csv_df = pd.DataFrame({
    "year": years,
    "commodity": commodities,
    "country": countries
})
empty_csv_df.to_csv("empty_CSVs.csv", index=False)
empty_csv_df
# df = pd.read_csv(f"{root_path_imports}/{list_files[0]}")

[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No columns to parse from file
[Error] No c

,year,commodity,country
0,2000,ALUMINIUM,Afghanistan
1,2000,ALUMINIUM,Albania
2,2000,ALUMINIUM,Angola
3,2000,ALUMINIUM,Armenia
4,2000,ALUMINIUM,Aruba
...,...,...,...
70559,2020,ZIRCONIUM,Uganda
70560,2020,ZIRCONIUM,United Rep. of Tanzania
70561,2020,ZIRCONIUM,Vanuatu
70562,2020,ZIRCONIUM,Venezuela


In [25]:
len(empty_csv_df["year"].unique())

21

## All the CSVs contain a row "WORLD" that is the aggregate of all the rows. We use such column to attach its value to all its respective rows.

In [26]:
# Lets attach the column "Total Imports" from reporterDesc where equals to "World"
world_rows = final_imports_df["partnerDesc"] == "World"
world_rows_df = final_imports_df[world_rows]

columns = ["period", "reporterISO", "cmdCode", "netWgt", "cifvalue", "primaryValue"]
world_rows_df = world_rows_df[columns]
world_rows_df.rename(
    columns={
        "netWgt": "netWgt_total_import",
        "cifvalue": "cifvalue_total_import",
        "primaryValue": "primaryValue_total_import"
        }, inplace=True)
world_rows_df

,period,reporterISO,cmdCode,netWgt_total_import,cifvalue_total_import,primaryValue_total_import
2,2000,DZA,2606,3.647900e+05,1.124830e+05,1.124830e+05
5,2000,AND,2606,4.200000e+01,5.000000e+01,5.000000e+01
7,2000,ATG,2606,1.000000e+00,1.300000e+01,1.300000e+01
11,2000,ARG,2606,3.650563e+07,6.665466e+06,6.665466e+06
13,2000,AUS,2606,9.817080e+06,1.927827e+06,1.927827e+06
...,...,...,...,...,...,...
405772,2020,USA,261510,2.396592e+07,3.623026e+07,3.623026e+07
405782,2020,UZB,261510,2.174000e+05,4.444400e+05,4.444400e+05
405795,2020,VNM,261510,4.814944e+06,7.323808e+06,7.323808e+06
405796,2020,ZMB,261510,6.300000e+04,2.694438e+04,2.694438e+04


In [27]:
pd.set_option('display.max_columns', None)

## Save result into a CSV

In [28]:
final_imports_df = pd.merge(final_imports_df, world_rows_df, how="left", on=["period", "reporterISO", "cmdCode"])
final_imports_df

,typeCode,freqCode,refPeriodId,refYear,refMonth,period,reporterCode,reporterISO,reporterDesc,flowCode,flowDesc,partnerCode,partnerISO,partnerDesc,partner2Code,partner2ISO,partner2Desc,classificationCode,classificationSearchCode,isOriginalClassification,cmdCode,cmdDesc,aggrLevel,isLeaf,customsCode,customsDesc,mosCode,motCode,motDesc,qtyUnitCode,qtyUnitAbbr,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,netWgt_total_import,cifvalue_total_import,primaryValue_total_import
0,C,A,20000101,2000,52,2000,12,DZA,Algeria,M,Import,528,NLD,Netherlands,0,W00,World,H1,HS,True,2606,Aluminium ores and concentrates,4,False,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,100400.0,False,-1,NaN,NaN,False,100400.0,False,NaN,False,37768.000,NaN,37768.000,0,False,False,364790.0,112483.000,112483.000
1,C,A,20000101,2000,52,2000,12,DZA,Algeria,M,Import,156,CHN,China,0,W00,World,H1,HS,True,2606,Aluminium ores and concentrates,4,False,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,263140.0,False,-1,NaN,NaN,False,263140.0,False,NaN,False,74064.000,NaN,74064.000,0,False,False,364790.0,112483.000,112483.000
2,C,A,20000101,2000,52,2000,12,DZA,Algeria,M,Import,0,W00,World,0,W00,World,H1,HS,True,2606,Aluminium ores and concentrates,4,False,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,364790.0,False,-1,NaN,NaN,False,364790.0,False,NaN,False,112483.000,NaN,112483.000,0,False,False,364790.0,112483.000,112483.000
3,C,A,20000101,2000,52,2000,12,DZA,Algeria,M,Import,724,ESP,Spain,0,W00,World,H1,HS,True,2606,Aluminium ores and concentrates,4,False,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,1250.0,False,-1,NaN,NaN,False,1250.0,False,NaN,False,651.000,NaN,651.000,0,False,False,364790.0,112483.000,112483.000
4,C,A,20000101,2000,52,2000,20,AND,Andorra,M,Import,724,ESP,Spain,0,W00,World,H1,HS,True,2606,Aluminium ores and concentrates,4,False,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,42.0,False,-1,NaN,NaN,False,42.0,False,NaN,False,50.000,NaN,50.000,0,False,False,42.0,50.000,50.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405796,C,A,20200101,2020,52,2020,894,ZMB,Zambia,M,Import,0,W00,World,0,W00,World,H5,HS,True,261510,Zirconium ores and concentrates,6,True,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,63000.0,False,8,kg,63000.0,False,63000.0,False,0.0,False,26944.376,0.0,26944.376,0,False,True,63000.0,26944.376,26944.376
405797,C,A,20200101,2020,52,2020,894,ZMB,Zambia,M,Import,380,ITA,Italy,0,W00,World,H5,HS,True,261510,Zirconium ores and concentrates,6,True,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,30000.0,False,8,kg,30000.0,False,30000.0,False,0.0,False,9204.315,0.0,9204.315,0,False,True,63000.0,26944.376,26944.376
405798,C,A,20200101,2020,52,2020,894,ZMB,Zambia,M,Import,710,ZAF,South Africa,0,W00,World,H5,HS,True,261510,Zirconium ores and concentrates,6,True,C00,TOTAL CPC,0,0,TOTAL MOT,8,kg,33000.0,False,8,kg,33000.0,False,33000.0,False,0.0,False,17740.061,0.0,17740.061,0,False,True,63000.0,26944.376,26944.376
405799,C,A,20200101,2020,52,2020,716,ZWE,Zimbabwe,M,Import,710,ZAF,South Africa,0,W00,World,H5,HS,True,261510,Zirconium ores and concentrates,6,True,C00,TOTAL CPC,0,0,TOTAL MOT,-1,NaN,0.0,True,8,kg,426196.5,False,0.0,True,0.0,False,845145.046,0.0,845145.046,6,False,True,0.0,845145.046,845145.046


In [29]:
final_imports_df.to_csv("../../data/imports_consolidated/imports_consolidated.csv", index=False)